In [101]:
import time
import os

import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains


In [102]:
def getReviews(reviews, dict_):
    for review in reviews:
        review_head = review.find("a", class_="fCitC").get_text()
        review = review.find("q", class_="XllAv H4 _a").get_text()
        dict_[review_head] = review

In [103]:
def getNegativeReviews(hotel_url):
    negativeReviews = {}
    
    driver = webdriver.Firefox()
    driver.get(hotel_url)
    
    driver.find_element_by_xpath(f".//label[contains(text(),'Poor')]").click()
    driver.find_element_by_xpath(f".//label[contains(text(),'Terrible')]").click()

    time.sleep(3)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    reviews = soup.find_all("div", class_="cWwQK MC R2 Gi z Z BB dXjiy")
    
    while True:        
        getReviews(reviews, negativeReviews)        
        
        pagination = soup.find("div", class_="ui_pagination is-centered")
        next_button = pagination.find("a", class_="ui_button nav next primary")
        
        if next_button:
            if "disabled" not in next_button['class']:
                driver.find_element_by_link_text("Next").click()                    
                time.sleep(3)       
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, 'html.parser')
                reviews = soup.find_all("div", class_="cWwQK MC R2 Gi z Z BB dXjiy")
                continue
            else:
                break

        break
    
    driver.close()
    return negativeReviews
    

In [104]:
hotel_url = 'https://www.tripadvisor.com/Hotel_Review-g293890-d3210915-Reviews-Hotel_Yambu-Kathmandu_Kathmandu_Valley_Bagmati_Zone_Central_Region.html'
negative = getNegativeReviews(hotel_url)

In [105]:
negativeDataframe = pd.DataFrame({'heading': negative.keys(),
                                  'reviews': negative.values()})

In [106]:
negativeDataframe

,heading,reviews
0,Dirty & cheated,"We stayed there for two night, after 15 days i..."
1,Very Terrible.,The toilet won't flush and the wifi usually do...
2,Not good,The room was dirty and the bathroom smelled un...
3,Cheep and Nasty,"Day One , no hot water we paid for a Deluxe ro..."
4,Stay somewhere else,"Dirty, needs a good clean. Sporadic wifi. Room..."
5,"Beware, your booking might be declined",It was a very bad experience for us. we have a...
6,Friendly Staff but Rough/Poor Infrastructure,Extremely friendly service. They helped us out...
7,Mould in bedroom and bathroom PLUS no water,"Yambu Hotel was a massive disappointment, so m..."
8,Low quality in good location.,Low quality in good location. There was No ele...
9,ganesh shakkarwar avoid this hotel in kathmandu,we were at yambu thamel from 6th to 8th feb 15...


In [107]:
def concatExcel():
    negativeExcel = pd.read_excel('negative.xlsx', header=0, index_col=0)
    negativeConcat = pd.concat([negativeExcel, negativeDataframe])
    negativeConcat.to_excel('negative.xlsx')

In [108]:
concatExcel()